In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
wuhan = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
chongqing = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/chongqing').first()).geometry()
chengdu = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/chengdu').first()).geometry()
nanjing = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/nanjing').first()).geometry()
kunming = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/kunming').first()).geometry()

city = ee.FeatureCollection([wuhan,chengdu,nanjing,kunming])
Map.addLayer(city, {}, "city")
Map.centerObject(city,6)

# city = ee.FeatureCollection([wuhan,chongqing,chengdu,nanjing,kunming])
# Map.addLayer(city, {}, "city")
# Map.centerObject(city,6)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

def ndvi_mndwi(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    class_ndvi_mndwi = ndvi_mndwi.lt(0).rename('class_ndvi_mndwi')
    class_mndwi = mndwi.gt(0).rename('class_mndwi')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(class_ndvi_mndwi).addBands(class_mndwi)

def area_after_clip(image):
    image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

In [5]:
# roi = wuhan
# landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#     .filterBounds(roi) \
#     .filterDate('2018-01-01', '2018-12-31') \
#     .map(maskL8sr) \
#     .map(ndvi_mndwi)
# Map.addLayer(landsat_images.mosaic().clip(roi),visParams,'landsat image')

In [6]:
def landsatCollection(roi):
    landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(roi) \
        .filterDate('2018-01-01', '2018-12-31') \
        .map(maskL8sr) \
        .map(ndvi_mndwi)
    return landsat_images

# 筛选得出采样影像

In [7]:
def sample_image(roi):
    image = landsatCollection(roi).mosaic().clip(roi)
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 30,
        'maxPixels': 1e14
    })
    # 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
    sampleImage = landsatCollection(roi).map(area_after_clip).filter(ee.Filter.gt('area',ee.Number(areas.get('B2')).divide(ee.Number(10))))
    return sampleImage

In [8]:
# roi = wuhan
# sam_image = sample_image(roi)
# dates = sam_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
# print(sam_image.size().getInfo())
# print(dates.getInfo())

In [9]:
def waterfrequency(roi):
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    water_layer = landsatCollection(roi).map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
    validPixel_layer = landsatCollection(roi).count().select('B3').clip(roi).rename('count')
    index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
    index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
    index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
    index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
    # JRC frequency
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    JRC_2018 = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
    JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
    # 交集数据部分
    permanentWaterExtent_one = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2)]).sum().eq(2)
    permanentWaterExtent = permanentWaterExtent_one.rename('permanent')
    landExtent_one = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0)]).sum().eq(2)     
    landExtent = landExtent_one.rename('land')    
    seasonWaterExtent = ee.ImageCollection([permanentWaterExtent_one,landExtent_one]).sum().remap([0,1],[1,0]).rename('waterclass').rename('season')
    return permanentWaterExtent.addBands(landExtent).addBands(seasonWaterExtent)

In [10]:
a = waterfrequency(wuhan)
print(a.getInfo())

{'type': 'Image', 'bands': [{'id': 'permanent', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'land', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'season', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [11]:
# wuhan = waterfrequency(roi)
# Map.addLayer(wuhan.select('permanent'),{'min':0,'max':1,'palette':['white','blue']},'permanent')
# Map.addLayer(wuhan.select('land'),{'min':0,'max':1,'palette':['white','blue']},'permanent')

In [12]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi','class_ndvi_mndwi','class_mndwi']
# 分类标签
label = 'waterclass'

def imageSample(image):
    waterFrequency = waterfrequency(roi)
    permanentWaterExtent = waterFrequency.select('permanent').rename('waterclass')
    seasonWaterExtent = waterFrequency.select('season').rename('waterclass')   
    landExtent = waterFrequency.select('land').rename('waterclass')
    permanent = permanentWaterExtent.clip(image.geometry()).selfMask()
    season = AWEIsh(image.clip(roi).updateMask(seasonWaterExtent)).select('AWEI').gt(0).rename('waterclass')
    land = landExtent.clip(image.geometry()).selfMask().remap([1],[0]).rename('waterclass')
    points_permanentwater = permanent.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 200,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    points_seasonalwater = season.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 200,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    points_land = land.sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': 100,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })
    all_points = points_permanentwater.merge(points_seasonalwater).merge(points_land)
    sample_points = image.select(bands).sampleRegions(**{
        'collection': all_points,
        'properties': [label],
        'scale': 30
    })
    return sample_points

In [13]:
# # 简介版
# roi = wuhan
# wuhan_pointsCollection = sample_image(roi).map(imageSample).flatten()

# roi = nanjing
# nanjing_pointsCollection = sample_image(roi).map(imageSample).flatten()

# roi = chengdu
# chengdu_pointsCollection = sample_image(roi).map(imageSample).flatten()

# roi = kunming
# kunming_pointsCollection = sample_image(roi).map(imageSample).flatten()

# pointsCollection = wuhan_pointsCollection.merge(nanjing_pointsCollection)
# pointsCollection2 = pointsCollection.merge(chengdu_pointsCollection)
# pointsCollection3 = pointsCollection2.merge(kunming_pointsCollection)
# print(pointsCollection3.size().getInfo())
# samplepoint_water = pointsCollection3.filter(ee.Filter.eq('waterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

In [14]:
roi = wuhan
# 总的样本点数
wuhan_pointsCollection = sample_image(roi).map(imageSample).flatten()

print('total sample number:{}'.format(wuhan_pointsCollection.size().getInfo()))
# 水体的样本点数
wuhan_samplepoint_water = wuhan_pointsCollection.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(wuhan_samplepoint_water.size().getInfo()))

total sample number:1637
water sample number:324


In [15]:
roi = nanjing
# 总的样本点数
nanjing_pointsCollection = sample_image(roi).map(imageSample).flatten()
print('total sample number:{}'.format(nanjing_pointsCollection.size().getInfo()))

# 水体的样本点数
nanjing_samplepoint_water = nanjing_pointsCollection.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(nanjing_samplepoint_water.size().getInfo()))

ProxyError: ('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
roi = chengdu
# 总的样本点数
chengdu_pointsCollection = sample_image(roi).map(imageSample).flatten()
print('total sample number:{}'.format(chengdu_pointsCollection.size().getInfo()))

# 水体的样本点数
chengdu_samplepoint_water = chengdu_pointsCollection.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(chengdu_samplepoint_water.size().getInfo()))

In [ ]:
roi = kunming
# 总的样本点数
kunming_pointsCollection = sample_image(roi).map(imageSample).flatten()
print('total sample number:{}'.format(kunming_pointsCollection.size().getInfo()))

# 水体的样本点数
kunming_samplepoint_water = kunming_pointsCollection.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(kunming_samplepoint_water.size().getInfo()))

In [ ]:
pointsCollection = wuhan_pointsCollection.merge(nanjing_pointsCollection)
pointsCollection2 = pointsCollection.merge(chengdu_pointsCollection)
pointsCollection3 = pointsCollection2.merge(kunming_pointsCollection)
# print(pointsCollection3.size().getInfo())

In [ ]:
print(pointsCollection3.first().getInfo())

In [ ]:
test_accuracy = pointsCollection3.errorMatrix('class_mndwi', 'waterclass')
print(test_accuracy.getInfo())

In [ ]:
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
samplepoint_water = pointsCollection3.filter(ee.Filter.eq('waterclass',0))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

In [ ]:
samplepoint_water = pointsCollection3.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# 训练样本，生成结果

In [ ]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi']
# 分类标签
label = 'waterclass'

trainedClassifier = ee.Classifier.smileRandomForest(20).train(pointsCollection3,label,bands)
#对Landsat-8进行分类
def training(image):
    return image.clip(roi).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')

In [ ]:
city = ee.FeatureCollection([wuhan,chengdu,nanjing,kunming])
# print(landsatCollection(city).size().getInfo())
roi = city
waterpixel_count = landsatCollection(city).map(training).sum()
# Map.addLayer(waterpixel_count,{},"waterPixelCount")

In [ ]:
basemap = ee.Image.constant(0).clip(city).rename('waterclass')
Map.addLayer(basemap,{'palette':['white']},"basemap")

validPixel = landsatCollection(city).count().select('B3').clip(roi).rename('count')
waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"frequency")

permanentwater = waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')
Map.addLayer(permanentwater,{'palette':['blue']},"permanent water")

In [ ]:
print(landsatCollection(city).size().getInfo())

# 制图

## 水体频率制图

In [ ]:
# 武汉
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(wuhan)', fontsize=20)

In [ ]:
# 成都
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [102.95, 30.0, 104.95, 31.5] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(chengdu)', fontsize=20)

In [ ]:
# 南京
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [118.3, 31.16, 119.3, 32.7] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(nanjing)', fontsize=20)

In [ ]:
# 昆明
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [102.1, 24.3, 103.7, 26.6] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'waterbody frequency in 2018(kunming)', fontsize=20)

## 永久性水体制图

In [ ]:
permanentwater_bi = waterfrequency.gte(0.75).select('frequency').rename('waterclass')

In [ ]:
# 武汉
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['CDCDCD','0000ff'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater_bi, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(wuhan)', fontsize=20)

In [ ]:
# 成都
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [102.95, 30.0, 104.95, 31.5]
vis = {'palette':['CDCDCD','0000ff'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater_bi, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(chengdu)', fontsize=20)

In [ ]:
# 南京
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [118.3, 31.16, 119.3, 32.7]
vis = {'palette':['CDCDCD','0000ff'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater_bi, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(nanjing)', fontsize=20)

In [ ]:
# 昆明
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [102.1, 24.3, 103.7, 26.6]
vis = {'palette':['CDCDCD','0000ff'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater_bi, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.3], linestyle=":")
ax.set_title(label = 'permanent surface water in 2018(kunming)', fontsize=20)

# 附录

In [ ]:
# Map.addLayer(sample_image.first().clip(roi),visParams,'image first')
permanent_image = sample_image.first().clip(roi).updateMask(permanentWaterExtent)
Map.addLayer(permanent_image,visParams,'permanent image')
permanent = permanentWaterExtent.updateMask(permanent_image.select('B2').gt(0))
Map.addLayer(permanent,{'palette':['white','blue'],'min':0,'max':1},'permanent')


season_image = sample_image.first().clip(roi).updateMask(seasonWaterExtent)
seasonal = seasonWaterExtent.updateMask(season_image.select('B2').gt(0))
Map.addLayer(seasonal,{'palette':['white','red'],'min':0,'max':1},'seasonal')

land_image = sample_image.first().clip(roi).updateMask(landExtent)
land = landExtent.updateMask(land_image.select('B2').gt(0)).remap([1],[0]).rename('waterclass')
Map.addLayer(land,{'palette':['white','cyan'],'min':0,'max':1},'land')

points_permanentwater = permanent.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print("permanent : {}".format(points_permanentwater.size().getInfo()))
points_seasonalwater = seasonal.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('seasonal : {}'.format(points_seasonalwater.size().getInfo()))

points_land = land.sample(**{
    'region': roi,
    'scale': 30,
    'numPixels': 200,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('land : {}'.format(points_land.size().getInfo()))

In [ ]:
all_points = points_permanentwater.merge(points_seasonalwater).merge(points_land)
print(all_points.size().getInfo())
print(all_points.filter(ee.Filter.eq('waterclass',1)).size().getInfo())

In [ ]:
per_wuhan = landsat_images.mosaic().updateMask(permanentWaterExtent)
Map.addLayer(per_wuhan,visParams,"per wuhan")

In [ ]:
dataset = ee.Image('JRC/GSW1_2/YearlyHistory/2018')

visualization = {
  'bands': ['waterClass'],
  'min': 0.0,
  'max': 3.0,
  'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
}
Map.addLayer(dataset.clip(roi), visualization, 'Water Class')

In [ ]:
# Maryland_waterclass显示
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4] 
vis = {'palette':['cccccc', '99d9ea', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(Maryland_waterclass, region=region, vis_params=vis)
# add a colorbar to the map using the visualization params we passed to the map
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# ticks=range(0,50,10)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency(Maryland) in 2018', fontsize=20)

# JRC_waterclass
from matplotlib import pyplot as plt
from geemap import cartoee
import numpy as np
import matplotlib
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '99d9ea', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(JRC_waterclass, region=region, vis_params=vis)
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency(JRC) in 2018', fontsize=20)

# index_waterclass
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '99d9ea', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(index_waterclass, region=region, vis_params=vis)

# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency(INDEX) in 2018', fontsize=20)

In [ ]:
# seasonWaterExtent
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '99d9ea'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(seasonWaterExtent, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'seasonal waterbody extent in 2018', fontsize=20)

# permanentWaterExtent
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '0000ff'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentWaterExtent, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'permanent waterbody extent in 2018', fontsize=20)

# landExtent
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['white','cccccc'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(landExtent, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'land extent in 2018', fontsize=20)